In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
%%time
adatas_pk = []
for ti in [6,8,10,12,14,17,21]:
    print(ti)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk.obs = adata_pk.obs.set_index('cell')
    print(adata_pk.var['peak'][:3].values)
    
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
# get cells based on RNA
# get matrix based on ATAC - CPM or CP10k

same_sign_alltime = []
qs_alltime = []
l2fc_alltime = []
for cond_order, obssub in adata.obs.groupby('cond_order'):
    # get sub
    adatasub = adata[obssub.index]
    adata_pk = adatas_pk[cond_order]
    
    # get A vs C 
    x0 = adatasub.obsm['pca_p17on'][...,0]
    # x_shuff
    x = x0[np.random.choice(len(x0), size=len(x0), replace=False)]
    cond_a = x < np.percentile(x,20)
    cond_b = np.logical_and(
                    x > np.percentile(x,40),
                    x < np.percentile(x,60),)
    cond_c = x > np.percentile(x,80)
    cells_a = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_a].obs.index.values])
    cells_b = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_b].obs.index.values])
    cells_c = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_c].obs.index.values])
    
    cells_atac = adata_pk.obs.index.values
    adata_pk_a = adata_pk[np.intersect1d(cells_a, cells_atac)]
    adata_pk_b = adata_pk[np.intersect1d(cells_b, cells_atac)]
    adata_pk_c = adata_pk[np.intersect1d(cells_c, cells_atac)]
    
    mat_a = np.log10(1+adata_pk_a.X[...]) # layers['lognorm'][...]
    mat_b = np.log10(1+adata_pk_b.X[...]) # layers['lognorm'][...]
    mat_c = np.log10(1+adata_pk_c.X[...]) # adatasub_c.layers['lognorm'][...]
    
    # DEGs
    lfc_ba = np.log2(10)*(np.mean(mat_b, axis=0) - np.mean(mat_a, axis=0))
    lfc_bc = np.log2(10)*(np.mean(mat_b, axis=0) - np.mean(mat_c, axis=0)) 
    ts, ps_ba = stats.ttest_ind(mat_b, mat_a)
    ts, ps_bc = stats.ttest_ind(mat_b, mat_c)
    rs, qs_ba, _, _ = multipletests(np.nan_to_num(ps_ba, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_bc, _, _ = multipletests(np.nan_to_num(ps_bc, nan=1).reshape(-1,), method='fdr_bh')
    
    # sig_cond
    same_sign = lfc_ba*lfc_bc > 0 # peak or valley; # same sign
    lfc = np.mean([lfc_ba, lfc_bc], axis=0) # only relevant if they have the same sign
    qs = np.minimum(qs_ba, qs_bc)
    
    cond_sig = np.all([
        same_sign,
        qs < 0.05, 
        np.abs(lfc) > np.log2(1.5),
    ], axis=0)
    num_sig = np.sum(cond_sig)
    print(cond_order, mat_a.shape, mat_b.shape, mat_c.shape, num_sig)
    
    # save this
    same_sign_alltime.append(same_sign)
    qs_alltime.append(qs)
    l2fc_alltime.append(lfc)
    
    # break

In [ ]:
same_sign_alltime = np.array(same_sign_alltime)
qs_alltime = np.array(qs_alltime)
l2fc_alltime = np.array(l2fc_alltime)

In [ ]:
# adata.uns['qs_avc'] = qs_avc
# adata.uns['l2fc_avc'] = l2fc_avc
# adata

In [ ]:
l2fc_th = np.log2(1.5)
alpha_th = 0.05
cond_sig    = np.all([same_sign_alltime > 0, qs_alltime < alpha_th, np.abs(l2fc_alltime) > l2fc_th], axis=0)
cond_sig_bp = np.all([same_sign_alltime > 0, qs_alltime < alpha_th, l2fc_alltime >  l2fc_th], axis=0)
cond_sig_bn = np.all([same_sign_alltime > 0, qs_alltime < alpha_th, l2fc_alltime < -l2fc_th], axis=0)

print('num B vs (A/C) DARs per cond:\t', cond_sig.sum(axis=1))
print('num B   per cond:\t',    cond_sig_bp.sum(axis=1))
print('num A/C per cond:\t',    cond_sig_bn.sum(axis=1))

In [ ]:
gene_sig_instances    = cond_sig.sum(axis=0)
gene_sig_instances_bp = cond_sig_bp.sum(axis=0)
gene_sig_instances_bn = cond_sig_bn.sum(axis=0)

instances, counts     = np.unique(gene_sig_instances, return_counts=True)
instances_bp, counts_bp = np.unique(gene_sig_instances_bp, return_counts=True)
instances_bn, counts_bn = np.unique(gene_sig_instances_bn, return_counts=True)

print('num B vs (A/C) DARs in num conds:\t',  counts   , np.any(cond_sig, axis=0).sum())
print('num B   DARs in num conds:\t',         counts_bp, np.any(cond_sig_bp, axis=0).sum())
print('num A/C DARs in num conds:\t',         counts_bn, np.any(cond_sig_bn, axis=0).sum())

In [ ]:
# %%time
# fout0 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/samesign_b_vs_ac_p6to21.txt'
# fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_b_vs_ac_p6to21.txt'
# fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_b_vs_ac_p6to21.txt'

# np.savetxt(fout0, same_sign_alltime)
# np.savetxt(fout1, qs_alltime)
# np.savetxt(fout2, l2fc_alltime)

In [ ]:
same_sign_alltime

In [ ]:
l2fc_alltime